# Shortest paths, diameter, and average path lengths

*October 26 2022*

This notebook explores two basic shortest path algorithms that you probably remember from your BSc courses. We use them to calculate the diameter and average shortest path lengths of empirical networks.

In [1]:
from collections import defaultdict

import numpy as np
from scipy import linalg

import pathpy as pp

We start by constructing a simple example for an undirected network, which we already know from the lecture.

In [2]:
n_undirected = pp.Network(directed=False)
n_undirected.add_edge('a', 'b')
n_undirected.add_edge('b', 'c')
n_undirected.add_edge('c', 'a')
n_undirected.add_edge('d', 'e')
n_undirected.add_edge('e', 'f')
n_undirected.add_edge('f', 'g')
n_undirected.add_edge('g', 'd')
n_undirected.add_edge('d', 'f')
n_undirected.add_edge('b', 'd')
n_undirected.plot()

# Dijkstra's algorithm for single-source shortest paths

In the lecture we mention Dikstra's algorithm to calculate the shortest from a given (single) source node to all other nodes. Here we provide a simple (and inefficient) implementation, which should only serve to refresh your memory of the algorithm.

The algorithm works as follows: Starting from the source node, we first set the distance to all other nodes to infinity. The distance to the source node itself is set to zero. We then add all nodes along with their distances to a set `Q` (which - for reasons of efficiency - is usually implemented as a priority queue). In each iteration of the algorithm, a node $u$ with minimal distance from the given source node is removed from `Q`. In the beginning, this is necessarily the source node, which is the only node with distance zero. In each iteration, the distance to all successors $v$ of node $u$ (i.e. nodes for which a link $(u,v)$ exists) is updated. If the old distance from the source node was larger than the distance from the source to $u$ plus one, we found a shorter path to $v$ that passes through node $u$. In the first step of the algorithm, this will update the distance of all neighbours of the source node to one. Repeating this loop will incrementally set the correct shortest path lengths along all paths starting from source. The algorithm is repeated until the set `Q` is empty. 

Whenever we update the distance of a node $v$, we can also store a pointer that points in the (opposite) direction of the shortest path from the source to node $v$. This will allow us to reconstruct all shortest paths and to visualise the shortest path structure of the network (for a single source) as a directed network.

In terms of computational complexity, we highlight that the implementation of the set `Q` is crucial. In the implementation below we simply use a dictionary to store the current distances of all nodes in `Q`. In each iteration we have to determine the key with minimal distance, which requires a linear search. This leads to a time complexity of $\mathcal{O}(n^2)$, where $n$ is the number of nodes in the network. An optimal implementation using a priority queue implemented as a Fibonacci heap has a worst-time complexity of $\mathcal{O}(n\log n+m)$, where $n$ is the number of nodes and $m$ is the number of edges.

In [3]:
def dijkstra(network: pp.Network, source: str):
    """Naive (and inefficient) implementation of Dijkstra's algorithm 
    for unweighted networks"""
    
    Q: dict = dict()
    dist: dict = dict()
    prev: dict = dict()

    # Initialise distances and shortest path pointer for all other nodes
    for v in network.nodes.uids:
        if v != source:
            dist[v] = np.inf
            prev[v] = None
        else:
            dist[source] = 0
        # add all nodes to Q
        Q[v] = dist[v]

    while Q: # Reeat until Q is empty

        # Find entry with minimal distance and remove from Q
        u = min(Q.keys(), key=(lambda k: Q[k])) # To do this more efficiently, we should use a priority queue
        del Q[u]

        for v in network.successors[u]:
            # Calculate new distance value
            # Here we could also use edge weights to calculate least cost paths -> Exercise 01
            new_dist = dist[u] + 1           
            
            # Update distance value and pointer if distance is smaller than previously known
            if new_dist < dist[v.uid]:
                dist[v.uid] = new_dist
                prev[v.uid] = u
                if v.uid in Q:
                    Q[v.uid] = new_dist

    return dist, prev

Let us try this for our undirected example network, using node `a` as the source node.

In [4]:
dist, prev = dijkstra(n_undirected, 'a')
print(dist)

{'a': 0, 'd': 2, 'g': 3, 'c': 1, 'b': 1, 'f': 3, 'e': 3}


We can use the pointer dictionary `prev` to generate a shortest path tree, i.e. an acyclic directed network that can be used to construct shortest paths from the given source node to any other node.

In [5]:
n_tree = pp.Network(directed=True)

for k, v in prev.items():
    if v != None:
        n_tree.add_edge(v, k)
n_tree.plot()

In the example above, we can use the tree to determine that the shortest path from node `a` to node `g` is $(a,b,d,g)$. `pathpy` conveniently comes with a function that returns the shortest path tree for a given network and source node:

In [6]:
pp.algorithms.shortest_paths.shortest_path_tree(network=n_undirected, source='a').plot()

Naturally, Dijkstra's algorithm works for undirected and directed networks:

In [7]:
n_directed = pp.Network(directed=True)
n_directed.add_edge('a', 'b')
n_directed.add_edge('b', 'c')
n_directed.add_edge('c', 'a')
n_directed.add_edge('d', 'e')
n_directed.add_edge('e', 'f')
n_directed.add_edge('f', 'g')
n_directed.add_edge('g', 'd')
n_directed.add_edge('d', 'f')
n_directed.add_edge('b', 'd')
n_directed.plot()

In [8]:
dist, prev = dijkstra(n_directed, 'a')
print(dist)

{'a': 0, 'd': 2, 'g': 4, 'c': 2, 'b': 1, 'f': 3, 'e': 3}


Instead of our own implementation of the Disktra algorithm, we can use the function `single_source_shortest_path` in `pathpy`. It computes both the distances to all other nodes (as a vector) as well as the corresponding shortest paths, i.e. we do not need to reconstruct shortest paths ourselves from the `prev` pointers.

In [9]:
dist, paths = pp.algorithms.shortest_paths.single_source_shortest_paths(n_directed, source='a')
print(dist)
print(paths)

[0. 1. 2. 2. 3. 3. 4.]
{'b': ('a', 'b'), 'c': ('a', 'b', 'c'), 'd': ('a', 'b', 'd'), 'e': ('a', 'b', 'd', 'e'), 'f': ('a', 'b', 'd', 'f'), 'g': ('a', 'b', 'd', 'f', 'g')}


The shortest paths listed above can be determined from the shortest path tree:

In [11]:
pp.algorithms.shortest_paths.shortest_path_tree(n_directed, source='a').plot()

# Floyd-Warshall algorithm for all-pairs shortest path distances

Disktra's algorithm computes shortest paths from a single source to all other nodes. To compute shortest paths between all pairs of nodes we can use the algorithm proposed by Floyd and Warshall. Again, we provide an implementation (and brief explanation) only to refresh your memory from BSc courses. The Floyd-Warshall algorithm works as follows: It uses a distance matrix that stores the shortest path distance from each node to every other node. Initially, this matrix is filled with $\inf$, except for the diagonal, which contains zeros. Entries corresponding to pairs of nodes connected by directed or undirected links are set to one. 

The algorithm iterates through all nodes $v$. For each node $v$, it checks all source and destination nodes $s$ and $d$, and tests whether the currently known distance from $s$ to $d$ is larger than the sum of the distances from $s$ to $v$ and from $v$ to $d$. If this is the case, it updates the distance (in which case we have found a shorter path). Once the nested for loop has finished, all entries in the matrix correspond to the shortest path distances.

In [13]:
def floyd_warshall(network):
    n = network.number_of_nodes()

    # Initialize distance matrix
    distance_matrix = np.zeros((n, n))
    distance_matrix.fill(np.inf)
    np.fill_diagonal(distance_matrix, 0)
    
    # set distances of pairs of nodes connected by (directed or undirected) edges
    for e in network.edges:
        distance_matrix[network.nodes.index[e.v.uid], network.nodes.index[e.w.uid]] = 1
        if not network.directed:
            distance_matrix[network.nodes.index[e.w.uid], network.nodes.index[e.v.uid]] = 1

    # update distances
    for v in network.nodes.uids:
        for s in network.nodes.uids:
            for d in network.nodes.uids:
                if distance_matrix[network.nodes.index[s],network.nodes.index[d]] > distance_matrix[network.nodes.index[s],network.nodes.index[v]] + distance_matrix[network.nodes.index[v], network.nodes.index[d]]:
                    distance_matrix[network.nodes.index[s],network.nodes.index[d]] = distance_matrix[network.nodes.index[s],network.nodes.index[v]] + distance_matrix[network.nodes.index[v], network.nodes.index[d]]
    return distance_matrix

We apply the algorithm to the undirected network from above. The vector in the first row corresponds to the distance vector that was returned by the Dijkstra algorithm for source node `a`. The fact that the distance matrix contains $\inf$ values shows that this directed network is not strongly connected, i.e. there are pairs of nodes for which no path exists.

In [14]:
m = floyd_warshall(n_directed)
print(m)

[[ 0.  1.  2.  2.  3.  3.  4.]
 [ 2.  0.  1.  1.  2.  2.  3.]
 [ 1.  2.  0.  3.  4.  4.  5.]
 [inf inf inf  0.  1.  1.  2.]
 [inf inf inf  3.  0.  1.  2.]
 [inf inf inf  2.  3.  0.  1.]
 [inf inf inf  1.  2.  2.  0.]]


It is easy to see that the complexity of this algorithm is in $O(n^3)$, where $n$ is the number of nodes. `pathpy` provides a function that returns the shortest path distance matrix for a given network, which is internally computed based on the Floyd-Warshall algorithm.

In [15]:
m = pp.algorithms.shortest_paths.distance_matrix(n_directed)
print(m)

[[ 0.  1.  2.  2.  3.  3.  4.]
 [ 2.  0.  1.  1.  2.  2.  3.]
 [ 1.  2.  0.  3.  4.  4.  5.]
 [inf inf inf  0.  1.  1.  2.]
 [inf inf inf  3.  0.  1.  2.]
 [inf inf inf  2.  3.  0.  1.]
 [inf inf inf  1.  2.  2.  0.]]


# Floyd-Warshall with path reconstruction

The implementation of the Floyd-Warshall algorithm above calculates the distance matrix, but does not return actual shortest paths. We can add this feature by adding a matrix that stores a next pointer for each pair of source and target nodes (i.e. since we compute all shortest paths for all source nodes we have a next pointer matrix rather than a next pointer vector). This next pointer is updated whenever a new shortest path between a pair of nodes is found.

In [16]:
def floyd_warshall_paths(network):
    n = network.number_of_nodes()

    # initialize distance matrix    
    distance_matrix = np.zeros((n, n))
    distance_matrix.fill(np.inf)
    np.fill_diagonal(distance_matrix, 0)

    # initialize next pointers to None
    next_pointer = defaultdict(lambda: dict())
    for s in network.nodes.uids:
        for d in network.nodes.uids:
            next_pointer[s][d] = None
    
    # set distances and next pointers for pairs of nodes connected by an edge
    for e in network.edges:
        distance_matrix[network.nodes.index[e.v.uid], network.nodes.index[e.w.uid]] = 1
        next_pointer[e.v.uid][e.w.uid] = e.w.uid
        if not network.directed:
            distance_matrix[network.nodes.index[e.w.uid], network.nodes.index[e.v.uid]] = 1
            next_pointer[e.w.uid][e.v.uid] = e.v.uid
    
    # update distances and next pointers
    for v in network.nodes.uids:
        for s in network.nodes.uids:
            for d in network.nodes.uids:
                if distance_matrix[network.nodes.index[s], network.nodes.index[d]] > distance_matrix[network.nodes.index[s], network.nodes.index[v]] + distance_matrix[network.nodes.index[v], network.nodes.index[d]]:
                    distance_matrix[network.nodes.index[s], network.nodes.index[d]] = distance_matrix[network.nodes.index[s], network.nodes.index[v]] + distance_matrix[network.nodes.index[v], network.nodes.index[d]]
                    next_pointer[s][d] = next_pointer[s][v]
    return distance_matrix, next_pointer

We can now write a function that uses the next pointers to construct the shortest path from any source to any destination node.

In [20]:
def construct_path(next_pointer, s, d):
    # no path exists
    if next_pointer[s][d] == None:
        return tuple()
    # reconstruct path by traversing matrix of next pointers
    path = tuple(s)
    x = s
    while x != d:
        x = next_pointer[x][d]
        path += (x,)
    return path

Let us try this in the undirected network from above:

In [21]:
n_undirected.plot()

In [22]:
m, next_pointer = floyd_warshall_paths(n_undirected)
construct_path(next_pointer, 'g', 'e')

('g', 'd', 'e')

In [23]:
construct_path(next_pointer, 'a', 'e')

('a', 'b', 'd', 'e')

`pathpy` provides a function that computes both the shortest path distances as well as all shortest paths between all pairs of nodes. In the resulting dictionary, entry `paths['g']['e']` contains a set of shortest paths from node g to node e.

In [24]:
paths, m = pp.algorithms.shortest_paths.all_shortest_paths(n_undirected)
paths

defaultdict(<function pathpy.algorithms.shortest_paths.all_shortest_paths.<locals>.<lambda>()>,
            {'a': defaultdict(set,
                         {'b': {('a', 'b')},
                          'c': {('a', 'c')},
                          'a': {('a',)},
                          'e': {('a', 'b', 'd', 'e')},
                          'f': {('a', 'b', 'd', 'f')},
                          'g': {('a', 'b', 'd', 'g')},
                          'd': {('a', 'b', 'd')}}),
             'b': defaultdict(set,
                         {'a': {('b', 'a')},
                          'c': {('b', 'c')},
                          'd': {('b', 'd')},
                          'e': {('b', 'd', 'e')},
                          'f': {('b', 'd', 'f')},
                          'g': {('b', 'd', 'g')},
                          'b': {('b',)}}),
             'c': defaultdict(set,
                         {'b': {('c', 'b')},
                          'a': {('c', 'a')},
                          'd': {(

This function additionally returns the distance matrix (here for the undirected network):

In [25]:
print(m)

[[0. 1. 1. 2. 3. 3. 3.]
 [1. 0. 1. 1. 2. 2. 2.]
 [1. 1. 0. 2. 3. 3. 3.]
 [2. 1. 2. 0. 1. 1. 1.]
 [3. 2. 3. 1. 0. 1. 2.]
 [3. 2. 3. 1. 1. 0. 1.]
 [3. 2. 3. 1. 2. 1. 0.]]


## Network diameter and average shortest path length

The calculation of shortest paths between all pairs of nodes is the basis to calculate the diameter and the average shortest path length of a network. We simply compute the maximum and average of all entries in the distance matrix calculated by the Floyd-Warshall algorithm.

In [27]:
print('Diameter =', np.max(m))
print('Average shortest path length =', np.mean(m))

Diameter = 3.0
Average shortest path length = 1.5918367346938775


`pathpy` provides functions to compute these two quantities:

In [29]:
print('Diameter =', pp.algorithms.shortest_paths.diameter(n_undirected))
print('Average shortest path length =', pp.algorithms.shortest_paths.avg_path_length(n_undirected, exclude_zero=False))

Diameter = 3.0
Average shortest path length = 1.5918367346938775


In [30]:
help(pp.algorithms.shortest_paths.avg_path_length)

Help on function avg_path_length in module pathpy.algorithms.shortest_paths:

avg_path_length(network: 'Network', weight: 'Union[str, bool, None]' = None, exclude_zero: 'bool' = True) -> 'float'
    Calculates the average shortest path length in directed or undirected
        networks, according to the definition
    
            <l> := \sum_{i 
    eq j} D[i,j]/(n (n-1))
    
        where n is the number of nodes and D is a matrix containing shortest pair
        distances for all node pairs i,j. The above definition holds for the
        default case where paths between node pairs (i,i) are excluded.
    
        .. note::
    
            Shortest path lengths are calculated using the implementation
            of the Floyd-Warshall algorithm in scipy.csgraph.
    
        Parameters
        ----------
        network : Network
    
            The :py:class:`Network` object that contains the network
    
        weighted : bool
    
            If True cheapest paths will be calcu

Note, however, that all call to those two functions will internally calculate the shortest paths between all pairs of nodes twice! For large networks, this can lead to significant overhead, and in this case it can be more efficient to calculate the distance matrix once and the use it to calculate diameter and average path length in a second step.

# Diameter and average path length in empirical data

We now compute the diameter and the average shortest path length of a number of empirical networks. We specifically use the tables `physicians`, `highschool`, `gentoo`, `lotr` from the database file.

In [33]:
n_highschool = pp.io.sql.read_network(db_file='../data/networks.db', sql='SELECT source, target FROM "highschool"', directed=False)
n_physicians = pp.io.sql.read_network(db_file='../data/networks.db', sql='SELECT source, target FROM "physicians"', directed=False)
n_lotr = pp.io.sql.read_network(db_file='../data/networks.db', sql='SELECT DISTINCT source, target FROM "lotr"', directed=False)
n_gentoo = pp.io.sql.read_network(db_file='../data/networks.db', sql='SELECT source, target FROM "gentoo"', directed=True)

[10-26 15:38:42: WARNING] 154 edges existed already and were not be considered. To capture those edges, consider creating a multiedge and/or directed network.
[10-26 15:38:42: WARNING] 175 edges existed already and were not be considered. To capture those edges, consider creating a multiedge and/or directed network.
[10-26 15:38:42: WARNING] 67 edges existed already and were not be considered. To capture those edges, consider creating a multiedge and/or directed network.


In [34]:
print("Diameter of highschool is ", pp.algorithms.shortest_paths.diameter(n_highschool))
print("Diameter of physicians is ", pp.algorithms.shortest_paths.diameter(n_physicians))
print("Diameter of gentoo is ", pp.algorithms.shortest_paths.diameter(n_gentoo))
print("Diameter of lotr is ", pp.algorithms.shortest_paths.diameter(n_lotr))

print("Avg. path length of highschool is ", pp.algorithms.shortest_paths.avg_path_length(n_highschool))
print("Avg. path length of physicians is ", pp.algorithms.shortest_paths.avg_path_length(n_physicians))
print("Avg. path length of gentoo is ", pp.algorithms.shortest_paths.avg_path_length(n_gentoo))
print("Avg. path length of lotr is ", pp.algorithms.shortest_paths.avg_path_length(n_lotr))

Diameter of highschool is  12.0
Diameter of physicians is  inf
Diameter of gentoo is  inf
Diameter of lotr is  inf
Avg. path length of highschool is  5.362745098039215
Avg. path length of physicians is  inf
Avg. path length of gentoo is  inf
Avg. path length of lotr is  inf


Except for the `highschool` data set, all networks are disconnected, which is why we have an infinite diameter and average shortest path length. In those networks it would be more reasonable to first calculate connected components that we can then analyse. We show how to do this in the next notebook.